<a href="https://colab.research.google.com/github/shubhamksingh1/RecommendationEngine/blob/main/ContentBasedRecommendationLSTM/LSTM_Recommendation_Computer_Product_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import pandas as pd
from keras.layers import Dense, Input, Embedding, Dropout, LSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, Concatenate
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data

In [ ]:
train_df = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/train.pickle")

In [ ]:
test_df = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/test.pickle")

In [ ]:
train_df.shape,test_df.shape

((234740, 5), (45582, 5))

In [ ]:
train_df.columns,test_df.columns

(Index(['Product1', 'Product2', 'Cocompared', 'Desc1', 'Desc2'], dtype='object'),
 Index(['Product1', 'Product2', 'Cocompared', 'Desc1', 'Desc2'], dtype='object'))

In [ ]:
train_df

,Product1,Product2,Cocompared,Desc1,Desc2
0,1,2,1,tp link mbps universal wifi flipkart com genui...,netgear ac dual band gigabit wi fi router flip...
1,1,3,1,tp link mbps universal wifi flipkart com genui...,zyxel nbg simultaneous dual band wireless ac f...
2,1,4,1,tp link mbps universal wifi flipkart com genui...,trendnet tew brm flipkart com genuine product ...
3,1,5,1,tp link mbps universal wifi flipkart com genui...,iball wra gt flipkart com genuine product day ...
4,1,6,1,tp link mbps universal wifi flipkart com genui...,zyxel flipkart com genuine product day replace...
...,...,...,...,...,...
234735,485,480,1,fit model dell poweredge tower server compatib...,xeon silver ghz core mb cache ram gb slot matr...
234736,485,481,1,fit model dell poweredge tower server compatib...,dell poweredge server security silver bezel in...
234737,485,482,1,fit model dell poweredge tower server compatib...,fit wide standard equipment install depth post...
234738,485,483,1,fit model dell poweredge tower server compatib...,upc weight lb sff gb gb brand dell see product...


In [ ]:
test_df

,Product1,Product2,Cocompared,Desc1,Desc2
0,1,2,1,zyxel amg flipkart com genuine product day rep...,iball wireless adsl router flipkart com genuin...
1,1,3,1,zyxel amg flipkart com genuine product day rep...,tp link archer ac dual band router flipkart co...
2,1,4,1,zyxel amg flipkart com genuine product day rep...,iball ib gx wireless router flipkart com genui...
3,1,5,1,zyxel amg flipkart com genuine product day rep...,trendnet tha router flipkart com genuine produ...
4,1,6,1,zyxel amg flipkart com genuine product day rep...,wi bridge adw flipkart com genuine product day...
...,...,...,...,...,...
45577,214,209,1,processor manufacturer intel processor type co...,mpn ddt pv vn compatible dell poweredge tower ...
45578,214,210,1,processor manufacturer intel processor type co...,product type consumer electronics item package...
45579,214,211,1,processor manufacturer intel processor type co...,compute node package dimension centimeter pack...
45580,214,212,1,processor manufacturer intel processor type co...,pwn pwn material metal compatible dell powered...


## Some Config value

### Train Set

In [ ]:
from collections import Counter
results = Counter()
train_df['Desc1'].str.lower().str.split().apply(results.update)
print(results)
print(len(results)) #max_features

Counter({'usb': 290400, 'laptop': 226996, 'power': 165044, 'replacement': 151008, 'light': 148104, 'product': 146652, 'battery': 135036, 'free': 131648, 'port': 118096, 'led': 118096, 'warranty': 115676, 'adapter': 115676, 'day': 102124, 'quality': 99220, 'shipping': 98736, 'delivery': 98252, 'genuine': 97768, 'guarantee': 97768, 'cash': 97768, 'high': 91476, 'skin': 90992, 'display': 85668, 'size': 81796, 'color': 81312, 'com': 76472, 'monitor': 76472, 'flipkart': 75504, 'technology': 71632, 'performance': 71148, 'price': 67760, 'portable': 65340, 'easy': 63404, 'speed': 60500, 'flexible': 60500, 'inch': 59532, 'cable': 59048, 'print': 58564, 'design': 57596, 'shape': 57596, 'scan': 55176, 'data': 54208, 'brand': 53724, 'samsung': 53724, 'rate': 50820, 'compatible': 50336, 'specification': 49852, 'time': 49852, 'gaming': 49852, 'pad': 49368, 'hdmi': 48884, 'fan': 48400, 'pc': 48400, 'type': 47432, 'mouse': 46464, 'resolution': 45012, 'model': 44528, 'mb': 44528, 'included': 44044, 'al

In [ ]:
len_lst_d = []
for i in range(train_df.shape[0]):
    val_len = len(train_df['Desc1'][i].split(" "))
    len_lst_d.append(val_len)

print(max(len_lst_d)) #maxlen

652


### Test Set

In [ ]:

test_results = Counter()
test_df['Desc1'].str.lower().str.split().apply(test_results.update)
print(test_results)
print(len(test_results)) #max_features

Counter({'usb': 52398, 'laptop': 47712, 'replacement': 29181, 'product': 28968, 'battery': 28968, 'port': 28329, 'power': 27690, 'free': 25773, 'adapter': 25347, 'warranty': 23643, 'quality': 19170, 'genuine': 18957, 'delivery': 18744, 'skin': 18744, 'day': 18531, 'cash': 18531, 'display': 18531, 'guarantee': 18318, 'shipping': 18318, 'light': 17679, 'performance': 17253, 'technology': 16401, 'monitor': 15975, 'color': 15762, 'price': 14484, 'samsung': 14271, 'inch': 14058, 'size': 13632, 'led': 13419, 'flipkart': 13206, 'com': 13206, 'type': 12993, 'high': 12993, 'data': 12780, 'document': 12780, 'speed': 12567, 'cable': 12354, 'easy': 12354, 'print': 12141, 'drive': 12141, 'shape': 11715, 'rate': 11502, 'hdmi': 11502, 'also': 11289, 'compatible': 11289, 'software': 11076, 'ac': 10863, 'pc': 10863, 'design': 10863, 'specification': 10650, 'included': 10650, 'ssd': 10437, 'model': 10224, 'mouse': 10224, 'scan': 10224, 'charge': 10011, 'screen': 10011, 'gaming': 9798, 'pad': 9798, 'time

In [ ]:
len_lst_test = []
for i in range(test_df.shape[0]):
    val_len = len(test_df['Desc1'][i].split(" "))
    len_lst_test.append(val_len)

print(max(len_lst_test)) #maxlen

646


### Since Model is trained on Train Set we will use Train Set Configurations

In [ ]:
## some config values 
embed_size = 300 # how big should each word vector be
max_features = 2908 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 652 # max number of words in a title to use

In [ ]:
train_df.isna().sum(),test_df.isna().sum()

(Product1      0
 Product2      0
 Cocompared    0
 Desc1         0
 Desc2         0
 dtype: int64, Product1      0
 Product2      0
 Cocompared    0
 Desc1         0
 Desc2         0
 dtype: int64)

## Data Preparation To Feed Model

In [ ]:

train_A = train_df["Desc1"].values

test_A = test_df["Desc1"].values

train_B = train_df["Desc2"].values

test_B = test_df["Desc2"].values

In [ ]:
train_A[:5],train_B[:5]

(array(['tp link mbps universal wifi flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'tp link mbps universal wifi flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'tp link mbps universal wifi flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'tp link mbps universal wifi flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'tp link mbps universal wifi flipkart com genuine product day replacement guarantee free shipping cash delivery'],
       dtype=object),
 array(['netgear ac dual band gigabit wi fi router flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'zyxel nbg simultaneous dual band wireless ac flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'trendnet tew brm flipkart com genuine product day replacement guarantee free shippi

In [ ]:
test_A[:5],test_B[:5]

(array(['zyxel amg flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'zyxel amg flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'zyxel amg flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'zyxel amg flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'zyxel amg flipkart com genuine product day replacement guarantee free shipping cash delivery'],
       dtype=object),
 array(['iball wireless adsl router flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'tp link archer ac dual band router flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'iball ib gx wireless router flipkart com genuine product day replacement guarantee free shipping cash delivery',
        'trendnet tha router flipkart com genuine product day replacement guarantee f

## Model 

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_A) + list(train_B))

In [ ]:
type(tokenizer)

keras_preprocessing.text.Tokenizer

### Pickle Write Read Function

In [ ]:
import pickle

In [ ]:
def write_pickle(title, data):
    pikd = open(title + '.pickle', 'wb')
    pickle.dump(data, pikd)
    pikd.close()
# Example usage:
# write_pickle('filename', data) 

In [ ]:
write_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/tokenizer_desc",tokenizer)

In [ ]:
def read_pickle_file(file):
    pikd = open(file,'rb')
    data = pickle.load(pikd)
    pikd.close()
    return data

In [ ]:
tokenizer_load = read_pickle_file('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/tokenizer_desc.pickle')

In [ ]:
print(tokenizer.word_counts)

OrderedDict([('tp', 14520), ('link', 28072), ('mbps', 26136), ('universal', 7744), ('wifi', 5808), ('flipkart', 151008), ('com', 152944), ('genuine', 195536), ('product', 293304), ('day', 204248), ('replacement', 302016), ('guarantee', 195536), ('free', 263296), ('shipping', 197472), ('cash', 195536), ('delivery', 196504), ('netgear', 10648), ('ac', 50336), ('dual', 23232), ('band', 16456), ('gigabit', 12584), ('wi', 10648), ('fi', 8712), ('router', 54208), ('zyxel', 6776), ('nbg', 968), ('simultaneous', 968), ('wireless', 60984), ('trendnet', 4840), ('tew', 2904), ('brm', 968), ('iball', 3872), ('wra', 968), ('gt', 2904), ('huawei', 5808), ('hg', 968), ('cubetek', 968), ('airmobi', 968), ('iplay', 968), ('music', 3872), ('ib', 1936), ('wrh', 968), ('home', 35816), ('tl', 7744), ('mr', 968), ('apple', 55176), ('hn', 968), ('intex', 968), ('wdr', 968), ('tenda', 7744), ('te', 5808), ('ah', 3872), ('asus', 17424), ('rt', 5808), ('cloud', 11616), ('adsl', 11616), ('modem', 11616), ('mobil

In [ ]:
print(tokenizer_load.word_counts)

OrderedDict([('tp', 14520), ('link', 28072), ('mbps', 26136), ('universal', 7744), ('wifi', 5808), ('flipkart', 151008), ('com', 152944), ('genuine', 195536), ('product', 293304), ('day', 204248), ('replacement', 302016), ('guarantee', 195536), ('free', 263296), ('shipping', 197472), ('cash', 195536), ('delivery', 196504), ('netgear', 10648), ('ac', 50336), ('dual', 23232), ('band', 16456), ('gigabit', 12584), ('wi', 10648), ('fi', 8712), ('router', 54208), ('zyxel', 6776), ('nbg', 968), ('simultaneous', 968), ('wireless', 60984), ('trendnet', 4840), ('tew', 2904), ('brm', 968), ('iball', 3872), ('wra', 968), ('gt', 2904), ('huawei', 5808), ('hg', 968), ('cubetek', 968), ('airmobi', 968), ('iplay', 968), ('music', 3872), ('ib', 1936), ('wrh', 968), ('home', 35816), ('tl', 7744), ('mr', 968), ('apple', 55176), ('hn', 968), ('intex', 968), ('wdr', 968), ('tenda', 7744), ('te', 5808), ('ah', 3872), ('asus', 17424), ('rt', 5808), ('cloud', 11616), ('adsl', 11616), ('modem', 11616), ('mobil

In [ ]:
print(tokenizer.word_counts ==tokenizer_load.word_counts)

True


In [ ]:
train_A = tokenizer.texts_to_sequences(train_A)

test_A = tokenizer.texts_to_sequences(test_A)

train_B = tokenizer.texts_to_sequences(train_B)

test_B = tokenizer.texts_to_sequences(test_B)

In [ ]:
print(train_A[:5],train_B[:5])

[[443, 250, 267, 671, 818, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [443, 250, 267, 671, 818, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [443, 250, 267, 671, 818, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [443, 250, 267, 671, 818, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [443, 250, 267, 671, 818, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16]] [[550, 125, 304, 412, 494, 551, 624, 114, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [757, 2036, 2037, 304, 412, 92, 125, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [927, 1263, 2038, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [1041, 2039, 1264, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [757, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16]]


In [ ]:
print(test_A[:5],test_B[:5])

[[757, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [757, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [757, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [757, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [757, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16]] [[1041, 92, 516, 114, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [443, 250, 2085, 125, 304, 412, 114, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [1041, 1494, 92, 114, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [927, 114, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16], [551, 1048, 27, 25, 17, 6, 13, 4, 18, 8, 15, 19, 16]]


In [ ]:
label_col = 'Cocompared'

In [ ]:
lst=[]
for x in train_A:
    lst.append(len(x))
for x in train_B:
    lst.append(len(x))
for x in test_A:
    lst.append(len(x))
for x in test_B:
    lst.append(len(x))
print(max(lst)) # We can check the maxlen this way also

652


In [ ]:
%%time
## Pad the sentences 
train_A = pad_sequences(train_A, maxlen=maxlen)
test_A = pad_sequences(test_A, maxlen=maxlen)

train_B = pad_sequences(train_B, maxlen=maxlen)
test_B = pad_sequences(test_B, maxlen=maxlen)


CPU times: user 4.93 s, sys: 816 ms, total: 5.74 s
Wall time: 5.17 s


In [ ]:

## Get the target values
train_y = train_df[label_col].values
test_y = test_df[label_col].values


In [ ]:
train_A[:5]

array([[ 0,  0,  0, ..., 15, 19, 16],
       [ 0,  0,  0, ..., 15, 19, 16],
       [ 0,  0,  0, ..., 15, 19, 16],
       [ 0,  0,  0, ..., 15, 19, 16],
       [ 0,  0,  0, ..., 15, 19, 16]], dtype=int32)

In [ ]:
train_y[:5],test_y[:5]

(array([1, 1, 1, 1, 1]), array([1, 1, 1, 1, 1]))

### Network 

In [ ]:
def create_lstm_nn():
    seq = Sequential()
    # embedding layer
    seq.add(Embedding(max_features, embed_size, trainable=True))
    # encode via bidirectional LSTM
    seq.add(Bidirectional(LSTM(64, return_sequences=True)))
    seq.add(GlobalMaxPool1D())
    # some dropout for regularization
    seq.add(Dropout(0.1))
    return seq

lstm_nn = create_lstm_nn()

In [ ]:
input_a = Input(shape=(maxlen,))
input_b = Input(shape=(maxlen,))

processed_a = lstm_nn(input_a)
processed_b = lstm_nn(input_b)

merged = Concatenate()([processed_a, processed_b])
merged = Dense(64, activation='elu')(merged)
merged = Dropout(0.1)(merged)
out = Dense(1, activation='sigmoid')(merged)

In [ ]:
model = Model(inputs = [input_a, input_b], outputs=out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mse', 'mae'])

print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 652)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 652)]        0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 128)          1059280     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 256)          0           sequential_2[0][0]         

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
esCallback  = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience= 2,
    verbose=1,
    mode="min",
    restore_best_weights=False,
)

In [ ]:
results = model.fit(x=[train_A, train_B], y=train_y, validation_data=([test_A, test_B], test_y),shuffle=True, batch_size=128, epochs=10,callbacks=[esCallback])

Epoch 1/10
1834/1834 [==============================] - 468s 252ms/step - loss: 0.2204 - mse: 0.0628 - mae: 0.1302 - val_loss: 0.3806 - val_mse: 0.0305 - val_mae: 0.0362
Epoch 2/10
1834/1834 [==============================] - 460s 251ms/step - loss: 0.0016 - mse: 2.2829e-04 - mae: 0.0013 - val_loss: 0.3510 - val_mse: 0.0332 - val_mae: 0.0373
Epoch 3/10
1834/1834 [==============================] - 460s 251ms/step - loss: 2.7209e-04 - mse: 1.9834e-05 - mae: 2.5896e-04 - val_loss: 0.4483 - val_mse: 0.0339 - val_mae: 0.0378
Epoch 4/10
1834/1834 [==============================] - 463s 253ms/step - loss: 2.7083e-04 - mse: 3.7801e-05 - mae: 1.5547e-04 - val_loss: 0.3602 - val_mse: 0.0298 - val_mae: 0.0345
Epoch 00004: early stopping


In [ ]:
from keras.models import load_model


In [ ]:

model.save('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/desc_model.h5')  # creates a HDF5 file 'my_model.h5'



In [ ]:
import math

In [ ]:
print(math.sqrt(results.history['val_mse'][-1]))

0.17268871459575214


In [ ]:
results.history['loss'][-1],results.history['val_loss'][-1],results.history['mse'][-1],results.history['val_mse'][-1]

(0.0008810031577013433,
 0.3602498769760132,
 0.00015327142318710685,
 0.02982139214873314)

In [ ]:
from keras.models import load_model
# returns a compiled model
# identical to the previous one
model_loaded = load_model('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/desc_model.h5')

In [ ]:
test_y_pred_loaded = model_loaded.predict([test_A, test_B], batch_size=128)[:, 0]

In [ ]:
test_y_pred_loaded[:10]

array([1.       , 1.       , 1.       , 1.       , 1.       , 0.9999999,
       1.       , 1.       , 1.       , 1.       ], dtype=float32)

In [ ]:
test_y[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
len(test_y_pred_loaded)

45582

In [ ]:
import numpy as np
from sklearn import metrics

In [ ]:
f_scores = []
for thresh in np.arange(0.1, 0.9, 0.01):
    thresh = np.round(thresh, 2)
    f_score = metrics.f1_score(test_y, (test_y_pred_loaded>thresh).astype(int))
    f_scores.append((f_score, thresh))
    
fmax, opt_thresh = max(f_scores)
print("F1 score at threshold {0} is {1}".format(opt_thresh, fmax))

F1 score at threshold 0.89 is 0.8743800162323022


## saving results

In [ ]:
train_y[:20]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
%%time
train_y_loaded = model_loaded.predict([train_A, train_B], batch_size=128)[:, 0]

CPU times: user 2min 57s, sys: 7.46 s, total: 3min 4s
Wall time: 3min 21s


In [ ]:
train_y_loaded[:20]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.], dtype=float32)

In [ ]:
train_y[-20:],train_y_loaded[-20:]

(array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([7.23615159e-08, 4.52717774e-09, 3.55902485e-09, 1.08636771e-08,
        1.01522296e-07, 4.69730566e-08, 7.55284368e-09, 9.99515891e-01,
        9.99440253e-01, 9.98750687e-01, 9.99603570e-01, 9.99336541e-01,
        9.99536395e-01, 9.99521613e-01, 9.99132335e-01, 9.99531031e-01,
        9.99463141e-01, 9.99650002e-01, 9.98307586e-01, 9.99558985e-01],
       dtype=float32))

In [ ]:
len(train_y),len(train_y_loaded)

(234740, 234740)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(train_y_loaded, train_y , squared=False)
rmse_test = mean_squared_error(test_y_pred_loaded, test_y , squared=False)

In [ ]:
rmse,rmse_test

(0.0003049100757022078, 0.1726886906236454)

In [ ]:
train_pred_list = list(train_y_loaded)

In [ ]:
test_pred_list = list(test_y_pred_loaded)

In [ ]:
train_df.shape

(234740, 5)

In [ ]:
train_df

,Product1,Product2,Cocompared,Desc1,Desc2
0,1,2,1,tp link mbps universal wifi flipkart com genui...,netgear ac dual band gigabit wi fi router flip...
1,1,3,1,tp link mbps universal wifi flipkart com genui...,zyxel nbg simultaneous dual band wireless ac f...
2,1,4,1,tp link mbps universal wifi flipkart com genui...,trendnet tew brm flipkart com genuine product ...
3,1,5,1,tp link mbps universal wifi flipkart com genui...,iball wra gt flipkart com genuine product day ...
4,1,6,1,tp link mbps universal wifi flipkart com genui...,zyxel flipkart com genuine product day replace...
...,...,...,...,...,...
234735,485,480,1,fit model dell poweredge tower server compatib...,xeon silver ghz core mb cache ram gb slot matr...
234736,485,481,1,fit model dell poweredge tower server compatib...,dell poweredge server security silver bezel in...
234737,485,482,1,fit model dell poweredge tower server compatib...,fit wide standard equipment install depth post...
234738,485,483,1,fit model dell poweredge tower server compatib...,upc weight lb sff gb gb brand dell see product...


In [ ]:
train_df['Pred_Sim_Desc'] = train_pred_list

In [ ]:
train_df

,Product1,Product2,Cocompared,Desc1,Desc2,Pred_Sim_Desc
0,1,2,1,tp link mbps universal wifi flipkart com genui...,netgear ac dual band gigabit wi fi router flip...,1.000000
1,1,3,1,tp link mbps universal wifi flipkart com genui...,zyxel nbg simultaneous dual band wireless ac f...,1.000000
2,1,4,1,tp link mbps universal wifi flipkart com genui...,trendnet tew brm flipkart com genuine product ...,1.000000
3,1,5,1,tp link mbps universal wifi flipkart com genui...,iball wra gt flipkart com genuine product day ...,1.000000
4,1,6,1,tp link mbps universal wifi flipkart com genui...,zyxel flipkart com genuine product day replace...,1.000000
...,...,...,...,...,...,...
234735,485,480,1,fit model dell poweredge tower server compatib...,xeon silver ghz core mb cache ram gb slot matr...,0.999531
234736,485,481,1,fit model dell poweredge tower server compatib...,dell poweredge server security silver bezel in...,0.999463
234737,485,482,1,fit model dell poweredge tower server compatib...,fit wide standard equipment install depth post...,0.999650
234738,485,483,1,fit model dell poweredge tower server compatib...,upc weight lb sff gb gb brand dell see product...,0.998308


In [ ]:
test_df['Pred_Sim_Desc'] = test_pred_list

In [ ]:
test_df

,Product1,Product2,Cocompared,Desc1,Desc2,Pred_Sim_Desc
0,1,2,1,zyxel amg flipkart com genuine product day rep...,iball wireless adsl router flipkart com genuin...,1.000000
1,1,3,1,zyxel amg flipkart com genuine product day rep...,tp link archer ac dual band router flipkart co...,1.000000
2,1,4,1,zyxel amg flipkart com genuine product day rep...,iball ib gx wireless router flipkart com genui...,1.000000
3,1,5,1,zyxel amg flipkart com genuine product day rep...,trendnet tha router flipkart com genuine produ...,1.000000
4,1,6,1,zyxel amg flipkart com genuine product day rep...,wi bridge adw flipkart com genuine product day...,1.000000
...,...,...,...,...,...,...
45577,214,209,1,processor manufacturer intel processor type co...,mpn ddt pv vn compatible dell poweredge tower ...,0.153597
45578,214,210,1,processor manufacturer intel processor type co...,product type consumer electronics item package...,0.791363
45579,214,211,1,processor manufacturer intel processor type co...,compute node package dimension centimeter pack...,0.479180
45580,214,212,1,processor manufacturer intel processor type co...,pwn pwn material metal compatible dell powered...,0.001518


In [ ]:
train_df.to_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/train_df_pred.pickle")

In [ ]:
test_df.to_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/test_df_pred.pickle")

## Similarity Matrix Creation

In [ ]:
train_matrix = train_df.pivot(index = 'Product1', columns = 'Product2', values = 'Pred_Sim_Desc').fillna(1)

In [ ]:
test_matrix = test_df.pivot(index = 'Product1', columns = 'Product2', values = 'Pred_Sim_Desc').fillna(1)

In [ ]:
train_matrix

Product2,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485
Product1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579666e-32,4.864781e-34,5.644788e-36,1.027249e-34,1.333724e-31,2.297293e-34,1.688673e-33,1.600078e-34,1.811896e-34,2.181758e-33,3.292680e-34,1.027474e-31,5.604585e-34,6.048084e-34
2,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.564303e-32,4.731936e-34,5.525861e-36,1.020795e-34,1.317723e-31,2.239168e-34,1.636781e-33,1.538006e-34,1.783560e-34,2.124837e-33,3.303474e-34,1.000377e-31,5.476050e-34,5.887687e-34
3,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.625533e-38,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.856306e-32,8.687044e-34,1.014115e-35,1.861882e-34,2.408566e-31,4.111272e-34,3.006346e-33,2.829300e-34,3.260408e-34,3.896621e-33,6.006973e-34,1.831935e-31,1.003855e-33,1.080221e-33
4,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000

In [ ]:
test_matrix

Product2,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214
Product1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,9.999700e-01,1.000000e+00,1.000000e+00,1.919366e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.926400e-01,1.000000e+00,2.356108e-20,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,2.858935e-08,3.054404e-08,7.426510e-08,3.085659e-13,3.469163e-08,9.479123e-08,1.879723e-07,1.492305e-08,6.300514e-08,9.795122e-07,1.849471e-09,4.558837e-14,4.735159e-07,1.212009e-07,2.427750e-07,8.231366e-38,0.000000e+00,2.314706e-38,0.000000e+00,2.211119e-38,3.666398e-37,1.513311e-28,3.479561e-38,0.000000e+00,0.000000e+00,1.885257e-37,0.000000e+00,1.751319e-34,4.458820e-38,9.904181e-38,3.651967e-21,0.000000e+00,0.000000e+00,4.215251e-38,4.067769e-31,2.019606e-18,1.972388e-30,2.215084e-33,5.077085e-29,1.345225e-24
2,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999995e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999998e-01,1.000000e+00,1.000000e+00,9.997638e-01,1.000000e+00,1.000000e+00,3.289825e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.539900e-01,1.000000e+00,3.590620e-21,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.053464e-08,1.041362e-08,1.873355e-08,6.961134e-14,1.357114e-08,3.725896e-08,6.037100e-08,4.637967e-09,2.355592e-08,3.147680e-07,4.604989e-10,9.438965e-15,1.494647e-07,3.959937e-08,8.490712e-08,1.265271e-38,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.280014e-38,1.613266e-29,0.000000e+00,0.000000e+00,0.000000e+00,2.689494e-38,0.000000e+00,2.606840e-35,0.000000e+00,1.462621e-38,5.332585e-22,0.000000e+00,0.000000e+00,0.000000e+00,4.312724e-32,2.201739e-19,2.104221e-31,2.347094e-34,7.186459e-30,1.230207e-25
3,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999993e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999996e-01,1.000000e+00,1.000000e+00,9.995807e-01,1.000000e+00,1.000000e+00,1.559989e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,8.993520e-01,1.000000e+00,1.740369e-21,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,4.823275e-09,4.692755e-09,7.450754e-09,2.990425e-14,6.315321e-09,1.737687e-08,2.700674e-08,1.947873e-09,1.068928e-08,1.358038e-07,1.859626e-10,4.141629e-15,6.470528e-08,1.742649e-08,3.815890e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.587941e-38,8.389138e-30,0.000000e+00,0.000000e+00,0.000000e+00,1.323456e-38,0.000000e+00,1.265683e-35,0.000000e+00,0.000000e+00,2.354031e-22,0.000000e+00,0.000000e+00,0.000000e+00,2.180552e-32,1.163639e-19,1.088471e-31,1.196512e-34,3.542496e-30,5.668661e-26
4,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999996e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,9.998505e-01,1.000000e+00,1.000000e+00,5.039004e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.690483e-01,1.000000e+00,5.748157e-21,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000

In [ ]:
train_matrix.to_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/train_sim_mat.pickle")

In [ ]:
test_matrix.to_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/test_sim_mat.pickle")

In [ ]:
train_matrix.to_csv("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/train_sim_mat.csv")

In [ ]:
test_matrix.to_csv("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/test_sim_mat.csv")

In [ ]:
train_matrix_loaded = pd.read_csv("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/train_sim_mat.csv")

In [ ]:
train_matrix_loaded

,Product1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485
0,1,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579666e-32,4.864781e-34,5.644788e-36,1.027249e-34,1.333724e-31,2.297293e-34,1.688673e-33,1.600078e-34,1.811896e-34,2.181758e-33,3.292680e-34,1.027474e-31,5.604585e-34,6.048084e-34
1,2,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.564303e-32,4.731936e-34,5.525861e-36,1.020795e-34,1.317723e-31,2.239168e-34,1.636781e-33,1.538006e-34,1.783560e-34,2.124837e-33,3.303474e-34,1.000377e-31,5.476050e-34,5.887687e-34
2,3,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.625533e-38,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.856306e-32,8.687044e-34,1.014115e-35,1.861882e-34,2.408566e-31,4.111272e-34,3.006346e-33,2.829300e-34,3.260408e-34,3.896621e-33,6.006973e-34,1.831935e-31,1.003855e-33,1.080221e-33
3,4,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.834201e-38,0.000000e+00,0.000000e+00,0.00000

In [ ]:
test_matrix_loaded = pd.read_csv("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/test_sim_mat.csv")

In [ ]:
test_matrix_loaded

,Product1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214
0,1,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,9.999700e-01,1.000000e+00,1.000000e+00,1.919366e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.926400e-01,1.000000e+00,2.356108e-20,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,2.858935e-08,3.054404e-08,7.426510e-08,3.085659e-13,3.469163e-08,9.479123e-08,1.879723e-07,1.492305e-08,6.300514e-08,9.795122e-07,1.849471e-09,4.558837e-14,4.735159e-07,1.212009e-07,2.427750e-07,8.231366e-38,0.000000e+00,2.314706e-38,0.000000e+00,2.211119e-38,3.666398e-37,1.513311e-28,3.479561e-38,0.000000e+00,0.000000e+00,1.885257e-37,0.000000e+00,1.751319e-34,4.458820e-38,9.904181e-38,3.651967e-21,0.000000e+00,0.000000e+00,4.215251e-38,4.067769e-31,2.019606e-18,1.972388e-30,2.215084e-33,5.077085e-29,1.345225e-24
1,2,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999995e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999998e-01,1.000000e+00,1.000000e+00,9.997638e-01,1.000000e+00,1.000000e+00,3.289825e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.539900e-01,1.000000e+00,3.590620e-21,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.053464e-08,1.041362e-08,1.873355e-08,6.961134e-14,1.357114e-08,3.725896e-08,6.037100e-08,4.637967e-09,2.355592e-08,3.147680e-07,4.604989e-10,9.438965e-15,1.494647e-07,3.959937e-08,8.490712e-08,1.265271e-38,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.280014e-38,1.613266e-29,0.000000e+00,0.000000e+00,0.000000e+00,2.689494e-38,0.000000e+00,2.606840e-35,0.000000e+00,1.462621e-38,5.332585e-22,0.000000e+00,0.000000e+00,0.000000e+00,4.312724e-32,2.201739e-19,2.104221e-31,2.347094e-34,7.186459e-30,1.230207e-25
2,3,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999993e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999996e-01,1.000000e+00,1.000000e+00,9.995807e-01,1.000000e+00,1.000000e+00,1.559989e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,8.993520e-01,1.000000e+00,1.740369e-21,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,4.823275e-09,4.692755e-09,7.450754e-09,2.990425e-14,6.315321e-09,1.737687e-08,2.700674e-08,1.947873e-09,1.068928e-08,1.358038e-07,1.859626e-10,4.141629e-15,6.470528e-08,1.742649e-08,3.815890e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.587941e-38,8.389138e-30,0.000000e+00,0.000000e+00,0.000000e+00,1.323456e-38,0.000000e+00,1.265683e-35,0.000000e+00,0.000000e+00,2.354031e-22,0.000000e+00,0.000000e+00,0.000000e+00,2.180552e-32,1.163639e-19,1.088471e-31,1.196512e-34,3.542496e-30,5.668661e-26
3,4,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999996e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,9.998505e-01,1.000000e+00,1.000000e+00,5.039004e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.690483e-01,1.000000e+00,5.748157e-21,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.296383e-08,1.324320e-08,2.549505e-08,1.016470e-13,1.648139e-08,4.554157e-08,7.901529e-08,5.81011

In [ ]:
ind_to_asset_id_train = dict(train_matrix_loaded['Product1'])
ind_to_asset_id_test = dict(test_matrix_loaded['Product1'])

In [ ]:
print(ind_to_asset_id_train)
print(ind_to_asset_id_test)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 58, 58: 59, 59: 60, 60: 61, 61: 62, 62: 63, 63: 64, 64: 65, 65: 66, 66: 67, 67: 68, 68: 69, 69: 70, 70: 71, 71: 72, 72: 73, 73: 74, 74: 75, 75: 76, 76: 77, 77: 78, 78: 79, 79: 80, 80: 81, 81: 82, 82: 83, 83: 84, 84: 85, 85: 86, 86: 87, 87: 88, 88: 89, 89: 90, 90: 91, 91: 92, 92: 93, 93: 94, 94: 95, 95: 96, 96: 97, 97: 98, 98: 99, 99: 100, 100: 101, 101: 102, 102: 103, 103: 104, 104: 105, 105: 106, 106: 107, 107: 108, 108: 109, 109: 110, 110: 111, 111: 112, 112: 113, 113: 114, 114: 115, 115: 116, 116: 117, 117: 118, 118: 119, 119: 120, 120: 121, 121: 12

In [ ]:
desc_sim_mat_train = train_matrix.values
desc_sim_mat_test = test_matrix.values

In [ ]:
desc_sim_mat_train.shape,desc_sim_mat_test.shape

((485, 485), (214, 214))

In [ ]:
desc_sim_mat_train

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.02747352e-31, 5.60458487e-34, 6.04808420e-34],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00037720e-31, 5.47604962e-34, 5.88768708e-34],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.83193530e-31, 1.00385491e-33, 1.08022136e-33],
       ...,
       [7.34484699e-31, 6.72285814e-31, 1.21479150e-30, ...,
        1.00000000e+00, 9.98741567e-01, 9.98648942e-01],
       [3.41376600e-33, 3.12682122e-33, 5.71018128e-33, ...,
        9.98125494e-01, 1.00000000e+00, 9.99494195e-01],
       [4.23548982e-33, 3.85825274e-33, 7.04919718e-33, ...,
        9.98307586e-01, 9.99558985e-01, 1.00000000e+00]])

In [ ]:
desc_sim_mat_test

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        2.21508411e-33, 5.07708476e-29, 1.34522527e-24],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        2.34709373e-34, 7.18645863e-30, 1.23020713e-25],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.19651217e-34, 3.54249611e-30, 5.66866140e-26],
       ...,
       [1.72161918e-32, 1.64086963e-33, 7.38363083e-34, ...,
        1.00000000e+00, 1.36952769e-06, 2.06201547e-03],
       [1.73771468e-29, 2.05890770e-30, 8.31785069e-31, ...,
        3.86771859e-07, 1.00000000e+00, 1.13268834e-04],
       [3.49831318e-23, 3.01068705e-24, 1.26081864e-24, ...,
        1.51844742e-03, 3.58274032e-04, 1.00000000e+00]])

In [ ]:
write_pickle('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/ind_to_asset_id_train', ind_to_asset_id_train)
write_pickle('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/ind_to_asset_id_test', ind_to_asset_id_test)

In [ ]:
write_pickle('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/desc_sim_mat_train',desc_sim_mat_train)
write_pickle('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/desc_sim_mat_test',desc_sim_mat_test)

In [ ]:
sim_mat_train = read_pickle_file('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/desc_sim_mat_train.pickle')

In [ ]:
sim_mat_train

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.02747352e-31, 5.60458487e-34, 6.04808420e-34],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00037720e-31, 5.47604962e-34, 5.88768708e-34],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.83193530e-31, 1.00385491e-33, 1.08022136e-33],
       ...,
       [7.34484699e-31, 6.72285814e-31, 1.21479150e-30, ...,
        1.00000000e+00, 9.98741567e-01, 9.98648942e-01],
       [3.41376600e-33, 3.12682122e-33, 5.71018128e-33, ...,
        9.98125494e-01, 1.00000000e+00, 9.99494195e-01],
       [4.23548982e-33, 3.85825274e-33, 7.04919718e-33, ...,
        9.98307586e-01, 9.99558985e-01, 1.00000000e+00]])

In [ ]:
sim_mat_test = read_pickle_file('/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/desc_sim_mat_test.pickle')

In [ ]:
sim_mat_test

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        2.21508411e-33, 5.07708476e-29, 1.34522527e-24],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        2.34709373e-34, 7.18645863e-30, 1.23020713e-25],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.19651217e-34, 3.54249611e-30, 5.66866140e-26],
       ...,
       [1.72161918e-32, 1.64086963e-33, 7.38363083e-34, ...,
        1.00000000e+00, 1.36952769e-06, 2.06201547e-03],
       [1.73771468e-29, 2.05890770e-30, 8.31785069e-31, ...,
        3.86771859e-07, 1.00000000e+00, 1.13268834e-04],
       [3.49831318e-23, 3.01068705e-24, 1.26081864e-24, ...,
        1.51844742e-03, 3.58274032e-04, 1.00000000e+00]])

In [ ]:
sim_mat_train.shape,sim_mat_test.shape

((485, 485), (214, 214))

In [ ]:
dict_map_train = read_pickle_file("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/ind_to_asset_id_train.pickle")
dict_map_test = read_pickle_file("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/ind_to_asset_id_test.pickle")


In [ ]:
print(dict_map_train)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 58, 58: 59, 59: 60, 60: 61, 61: 62, 62: 63, 63: 64, 64: 65, 65: 66, 66: 67, 67: 68, 68: 69, 69: 70, 70: 71, 71: 72, 72: 73, 73: 74, 74: 75, 75: 76, 76: 77, 77: 78, 78: 79, 79: 80, 80: 81, 81: 82, 82: 83, 83: 84, 84: 85, 85: 86, 86: 87, 87: 88, 88: 89, 89: 90, 90: 91, 91: 92, 92: 93, 93: 94, 94: 95, 95: 96, 96: 97, 97: 98, 98: 99, 99: 100, 100: 101, 101: 102, 102: 103, 103: 104, 104: 105, 105: 106, 106: 107, 107: 108, 108: 109, 109: 110, 110: 111, 111: 112, 112: 113, 113: 114, 114: 115, 115: 116, 116: 117, 117: 118, 118: 119, 119: 120, 120: 121, 121: 12

In [ ]:
print(dict_map_test)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 58, 58: 59, 59: 60, 60: 61, 61: 62, 62: 63, 63: 64, 64: 65, 65: 66, 66: 67, 67: 68, 68: 69, 69: 70, 70: 71, 71: 72, 72: 73, 73: 74, 74: 75, 75: 76, 76: 77, 77: 78, 78: 79, 79: 80, 80: 81, 81: 82, 82: 83, 83: 84, 84: 85, 85: 86, 86: 87, 87: 88, 88: 89, 89: 90, 90: 91, 91: 92, 92: 93, 93: 94, 94: 95, 95: 96, 96: 97, 97: 98, 98: 99, 99: 100, 100: 101, 101: 102, 102: 103, 103: 104, 104: 105, 105: 106, 106: 107, 107: 108, 108: 109, 109: 110, 110: 111, 111: 112, 112: 113, 113: 114, 114: 115, 115: 116, 116: 117, 117: 118, 118: 119, 119: 120, 120: 121, 121: 12

## Testing

### Train Data

In [ ]:
import pandas as pd
import pickle

In [ ]:
asset = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/df_train.pickle")

In [ ]:
asset

,product_name,description,category,cleaned_description,product_id
0,tp-link 300 mbps universal wifi,tp-link 300 mbps universal wifi only for rs....,router,tp link mbps universal wifi flipkart com genui...,1
1,netgear r6220 ac1200 dual band gigabit wi-fi r...,netgear r6220 ac1200 dual band gigabit wi-fi...,router,netgear ac dual band gigabit wi fi router flip...,2
2,zyxel nbg 6503 simultaneous dual-band wireless...,zyxel nbg 6503 simultaneous dual-band wirele...,router,zyxel nbg simultaneous dual band wireless ac f...,3
3,trendnet tew-721brm,trendnet tew-721brm only for rs. 1920 from f...,router,trendnet tew brm flipkart com genuine product ...,4
4,iball wra300n3gt,iball wra300n3gt only for rs. 1999 from flip...,router,iball wra gt flipkart com genuine product day ...,5
...,...,...,...,...,...
480,"dell poweredge r510, r520, r720, r720xd, r730,...",dell poweredge server security silver beze...,servers,dell poweredge server security silver bezel in...,481
481,raising electronics 15u 4 post open frame 19in...,"1.fit 19"" wide standard equipment. install...",servers,fit wide standard equipment install depth post...,482
482,dell 3080 sff i5 10500 16gb 256gb (0f7vx),. upc: 884116372646 weight: 13.600 lbs 3080...,servers,upc weight lb sff gb gb brand dell see product...,483
483,"dell poweredge r530, r730, r730, r540, r740, r...",dell poweredge 2u sliding ready rail kit c...,servers,dell poweredge sliding ready rail kit compatib...,484


In [ ]:
id_list = asset['product_id'].tolist()
product_name_list = asset['product_name'].tolist()
description_list = asset['description'].tolist()
domain_list = asset['category'].tolist()

In [ ]:
asset.columns

Index(['product_name', 'description', 'category', 'cleaned_description',
       'product_id'],
      dtype='object')

#### Recommendation

In [ ]:
# Function that takes in asset title as input and outputs most similar asset
def get_recommendations_test(asset_id,n=5):
    
    # title = title.lower()
    
    ##READING ASSET CSV DATAFRAME PICKLE FILE
    asset = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/df_train.pickle")
    input_asset_id = asset_id
    
    # Reset index of main DataFrame and construct reverse mapping as before
    asset = asset.reset_index(drop=True)
    indices = pd.Series(asset.index, index=asset['product_id'])

    # Reading Similarity Matrix
    
    sim_desc = read_pickle_file("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/desc_sim_mat_train.pickle") 

    # Get the index of the article that matches the product_id
    idx = indices[input_asset_id]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(sim_desc[idx]))

    # Sort the article based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar article
    sim_scores = sim_scores[1:n+1]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]
    sim_percentage = ["{:.2f}".format(i[1]*100) for i in sim_scores]
    
    recommendation  = []
    for i in range(len(article_indices)):
        dict_reco = {}
        reco_dict = {}
        reco_dict['Domain'] = asset['category'][article_indices[i]]
        reco_dict['asset_description'] = asset['description'][article_indices[i]]
        reco_dict['asset_id'] = asset['product_id'][article_indices[i]]
        reco_dict['asset_title'] = asset['product_name'][article_indices[i]]
        recommendation.append(reco_dict)
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['product_id'][article_indices[i]]) + " Recommending Percentage : "+str(sim_percentage[i])+ "% asset_title " + str(asset['product_name'][article_indices[i]])+ " asset_description " + str(asset['description'][article_indices[i]])+ " Asset_Domain " + str(asset['category'][article_indices[i]])+"\n")
    return recommendation

In [ ]:
id = id_list[11]
domain = domain_list[11]
title = product_name_list[11]
desc = description_list[11]
print(id)
print(domain)
print(title)
print(desc)

12
router
intex w150d
  intex w150d only for rs. 1149 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!


In [ ]:
%%time
out_list = get_recommendations_test(id,5)

1th recommendation  asset_id 2 Recommending Percentage : 100.00% asset_title netgear r6220 ac1200 dual band gigabit wi-fi router asset_description   netgear r6220 ac1200 dual band gigabit wi-fi router only for rs. 5199 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery! Asset_Domain router

2th recommendation  asset_id 3 Recommending Percentage : 100.00% asset_title zyxel nbg 6503 simultaneous dual-band wireless ac750 asset_description   zyxel nbg 6503 simultaneous dual-band wireless ac750 only for rs. 3000 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery! Asset_Domain router

3th recommendation  asset_id 4 Recommending Percentage : 100.00% asset_title trendnet tew-721brm asset_description   trendnet tew-721brm only for rs. 1920 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery! Asset_Domain router

4th recommendation  asset_i

In [ ]:
out_list

[{'Domain': 'router',
  'asset_description': '  netgear r6220 ac1200 dual band gigabit wi-fi router only for rs. 5199 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!',
  'asset_id': 2,
  'asset_title': 'netgear r6220 ac1200 dual band gigabit wi-fi router'},
 {'Domain': 'router',
  'asset_description': '  zyxel nbg 6503 simultaneous dual-band wireless ac750 only for rs. 3000 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!',
  'asset_id': 3,
  'asset_title': 'zyxel nbg 6503 simultaneous dual-band wireless ac750'},
 {'Domain': 'router',
  'asset_description': '  trendnet tew-721brm only for rs. 1920 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!',
  'asset_id': 4,
  'asset_title': 'trendnet tew-721brm'},
 {'Domain': 'router',
  'asset_description': '  iball wra300n3gt only for rs. 1999 from flipkart.com. only genuine pr

### Test Data

In [ ]:
asset_test = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/df_test.pickle")

In [ ]:
asset_test

,product_name,description,category,cleaned_description,product_id
0,zyxel amg1312-t10b,zyxel amg1312-t10b only for rs. 2425 from fl...,router,zyxel amg flipkart com genuine product day rep...,1
1,iball 300m wireless-n adsl2+ 3g router,iball 300m wireless-n adsl2+ 3g router only ...,router,iball wireless adsl router flipkart com genuin...,2
2,tp-link archer c20 ac750 dual band router,tp-link archer c20 ac750 dual band router on...,router,tp link archer ac dual band router flipkart co...,3
3,iball ib-w4gx150n 4g wireless n router,iball ib-w4gx150n 4g wireless n router only ...,router,iball ib gx wireless router flipkart com genui...,4
4,trendnet tha-101 n300 router,trendnet tha-101 n300 router only for rs. 49...,router,trendnet tha router flipkart com genuine produ...,5
...,...,...,...,...,...
209,lenovo dcg x8 x8 x8ml2 pcie fh riser1 kit,product type:consumer electronics item pac...,servers,product type consumer electronics item package...,210
210,lenovo system 9532e5u ibm flex x240 m5 2xxeon ...,. m5 compute node package dimensions: 61 h ...,servers,compute node package dimension centimeter pack...,211
211,bestparts new 2u sliding ready rail kit compat...,p/n: 0pwn3 00pwn3 h4x6x 0h4x6x * material:...,servers,pwn pwn material metal compatible dell powered...,212
212,hpe 873830-s01 proliant microserver gen10 ultr...,. full product specifications: click to vie...,servers,full product specification click view number p...,213


In [ ]:

id_list_test = asset_test['product_id'].tolist()
product_name_list_test = asset_test['product_name'].tolist()
description_list_test = asset_test['description'].tolist()
domain_list_test = asset_test['category'].tolist()

#### Recommendation

In [ ]:
# Function that takes in asset title as input and outputs most similar asset
def get_recommendations_test_df(asset_id,n=5):
    
    # title = title.lower()
    
    ##READING ASSET CSV DATAFRAME PICKLE FILE
    asset = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/df_test.pickle")
    input_asset_id = asset_id
    
    # Reset index of main DataFrame and construct reverse mapping as before
    asset = asset.reset_index(drop=True)
    indices = pd.Series(asset.index, index=asset['product_id'])

    # Reading Similarity Matrix
    
    sim_desc = read_pickle_file("/content/drive/MyDrive/Content_Recommendation/LSTM_Similarity/Computer_Product/desc_sim_mat_test.pickle") 

    # Get the index of the article that matches the product_id
    idx = indices[input_asset_id]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(sim_desc[idx]))

    # Sort the article based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar article
    sim_scores = sim_scores[1:n+1]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]
    sim_percentage = ["{:.2f}".format(i[1]*100) for i in sim_scores]
    
    recommendation  = []
    for i in range(len(article_indices)):
        dict_reco = {}
        reco_dict = {}
        reco_dict['Domain'] = asset['category'][article_indices[i]]
        reco_dict['asset_description'] = asset['description'][article_indices[i]]
        reco_dict['asset_id'] = asset['product_id'][article_indices[i]]
        reco_dict['asset_title'] = asset['product_name'][article_indices[i]]
        recommendation.append(reco_dict)
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['product_id'][article_indices[i]]) + " Recommending Percentage : "+str(sim_percentage[i])+ "% asset_title " + str(asset['product_name'][article_indices[i]])+ " asset_description " + str(asset['description'][article_indices[i]])+ " Asset_Domain " + str(asset['category'][article_indices[i]])+"\n")
    return recommendation

In [ ]:
id_test = id_list_test[11]
domain_test = domain_list_test[11]
title_test = product_name_list_test[11]
desc_test = description_list_test[11]
print(id_test)
print(domain_test)
print(title_test)
print(desc_test)

12
router
apple airport express base station wireless router
  apple airport express base station wireless router only for rs. 6800 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!


In [ ]:
%%time
out_list_test = get_recommendations_test_df(id_test,5)

1th recommendation  asset_id 9 Recommending Percentage : 100.00% asset_title isun isn v01 asset_description   isun isn v01 only for rs. 1599 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery! Asset_Domain router

2th recommendation  asset_id 23 Recommending Percentage : 100.00% asset_title tenda 3g150b asset_description   tenda 3g150b only for rs. 3000 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery! Asset_Domain router

3th recommendation  asset_id 1 Recommending Percentage : 100.00% asset_title zyxel amg1312-t10b asset_description   zyxel amg1312-t10b only for rs. 2425 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery! Asset_Domain router

4th recommendation  asset_id 21 Recommending Percentage : 100.00% asset_title merlin wifi n repeater asset_description   merlin wifi n repeater only for rs. 2195 from flipkart.com. only 

In [ ]:
out_list_test

[{'Domain': 'router',
  'asset_description': '  isun isn v01 only for rs. 1599 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!',
  'asset_id': 9,
  'asset_title': 'isun isn v01'},
 {'Domain': 'router',
  'asset_description': '  tenda 3g150b only for rs. 3000 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!',
  'asset_id': 23,
  'asset_title': 'tenda 3g150b'},
 {'Domain': 'router',
  'asset_description': '  zyxel amg1312-t10b only for rs. 2425 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!',
  'asset_id': 1,
  'asset_title': 'zyxel amg1312-t10b'},
 {'Domain': 'router',
  'asset_description': '  merlin wifi n repeater only for rs. 2195 from flipkart.com. only genuine products. 30 day replacement guarantee. free shipping. cash on delivery!',
  'asset_id': 21,
  'asset_title': 'merlin wifi n repeater'},
 {'Domain': 'route

In [ ]:
id_test = id_list_test[-1]
domain_test = domain_list_test[-1]
title_test = product_name_list_test[-1]
desc_test = description_list_test[-1]
print(id_test)
print(domain_test)
print(title_test)
print(desc_test)

214
servers
dell precision 7000 7550 15.6" mobile workstation - full hd - 1920 x 1080 - intel core i7 (10th gen) i7-10750h hexa-core (6 core) 2.60 ghz - 16 gb ram - 256 gb ssd - aluminum titan gray - windows
 .   processor manufacturer: intel processor type: core i7 processor generation: 10th gen processor model: i7-10750h processor core: hexa-core (6 core)  see more product details 


In [ ]:
%%time
out_list_test = get_recommendations_test_df(id_test,5)

1th recommendation  asset_id 196 Recommending Percentage : 97.36% asset_title epson network interface unit - document scanner accessory (b12b808441) asset_description  .   product type: scanner package dimensions: 17.018 h x 25.4 l x 17.272 w (centimetres) package weight: 2.1 pounds country of origin : china  Asset_Domain scanners

2th recommendation  asset_id 210 Recommending Percentage : 79.14% asset_title lenovo dcg x8 x8 x8ml2 pcie fh riser1 kit asset_description     product type:consumer electronics item package dimension:9.0 inches l x 13.5 inches w x 6.5 inches h item package weight:1.8 pounds country of origin: china  Asset_Domain servers

3th recommendation  asset_id 77 Recommending Percentage : 77.80% asset_title karp ultra bright 28 leds usb lights (white) led light- wht-4 usb led light asset_description karp ultra bright 28 leds usb lights (white) led light- wht-4 usb led light (white)
                         price: rs. 599
				
			ultra bright 28 leds usb lights for noteb

In [ ]:
out_list_test

[{'Domain': 'scanners',
  'asset_description': ' .   product type: scanner package dimensions: 17.018 h x 25.4 l x 17.272 w (centimetres) package weight: 2.1 pounds country of origin : china ',
  'asset_id': 196,
  'asset_title': 'epson network interface unit - document scanner accessory (b12b808441)'},
 {'Domain': 'servers',
  'asset_description': '    product type:consumer electronics item package dimension:9.0 inches l x 13.5 inches w x 6.5 inches h item package weight:1.8 pounds country of origin: china ',
  'asset_id': 210,
  'asset_title': 'lenovo dcg x8 x8 x8ml2 pcie fh riser1 kit'},
 {'Domain': 'led light',
  'asset_description': 'karp ultra bright 28 leds usb lights (white) led light- wht-4 usb led light (white)\n                         price: rs. 599\n\t\t\t\t\n\t\t\tultra bright 28 leds usb lights for notebook computer laptop pc lighting portable mini lamps.features: new flexible bright mini 28 led usb light computer lamp for notebook computer pc this is new facial hair epi

In [ ]:
id_test = id_list_test[70]
domain_test = domain_list_test[70]
title_test = product_name_list_test[70]
desc_test = description_list_test[70]
print(id_test)
print(domain_test)
print(title_test)
print(desc_test)

71
led light
rrp lamp for computer keyboard tslplt02 usb led light
rrp lamp for computer keyboard tslplt02 usb led light (green)
                         price: rs. 98
				
			portable usb led light from portronics is a battery free usb light that is perfect to be used with laptops, adapters and power banks. it is very compact in size and flexible hence can fit into any position based on use and utility. the power consumption is very limited but is able to light the whole area around.
portable usb led light from portronics is a battery free usb light that is perfect to be used with laptops, adapters and power banks. it is very compact in size and flexible hence can fit into any position based on use and utility. the power consumption is very limited but is able to light the whole area around.


In [ ]:
%%time
out_list_test = get_recommendations_test_df(id_test,5)

1th recommendation  asset_id 78 Recommending Percentage : 99.96% asset_title himtek light led 1 usb led light asset_description   himtek light led 1 usb led light for rs.119 online. himtek light led 1 usb led light at best prices with free shipping & cash on delivery. only genuine products. 30 day replacement guarantee. Asset_Domain led light

2th recommendation  asset_id 83 Recommending Percentage : 99.95% asset_title himtek 9o v-99 usb led light asset_description   himtek 9o v-99 usb led light for rs.119 online. himtek 9o v-99 usb led light at best prices with free shipping & cash on delivery. only genuine products. 30 day replacement guarantee. Asset_Domain led light

3th recommendation  asset_id 88 Recommending Percentage : 99.95% asset_title instella bright warm white laptop or any usb led light asset_description   instella bright warm white laptop or any usb led light for rs.91 online. instella bright warm white laptop or any usb led light at best prices with free shipping & cash

In [ ]:
out_list_test

[{'Domain': 'led light',
  'asset_description': '  himtek light led 1 usb led light for rs.119 online. himtek light led 1 usb led light at best prices with free shipping & cash on delivery. only genuine products. 30 day replacement guarantee.',
  'asset_id': 78,
  'asset_title': 'himtek light led 1 usb led light'},
 {'Domain': 'led light',
  'asset_description': '  himtek 9o v-99 usb led light for rs.119 online. himtek 9o v-99 usb led light at best prices with free shipping & cash on delivery. only genuine products. 30 day replacement guarantee.',
  'asset_id': 83,
  'asset_title': 'himtek 9o v-99 usb led light'},
 {'Domain': 'led light',
  'asset_description': '  instella bright warm white laptop or any usb led light for rs.91 online. instella bright warm white laptop or any usb led light at best prices with free shipping & cash on delivery. only genuine products. 30 day replacement guarantee.',
  'asset_id': 88,
  'asset_title': 'instella bright warm white laptop or any usb led light